# SITK Wrapper for Keras models to enable Gridsearch, does not run because of the callbacks

In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *


# define GPU id to use
# 0 = 1080 Bus ID 2
# 1 = Titan Bus ID 131
# 2 = Titan Bus ID 132
GPU_IDS = '0,1'
current_gpu = choose_gpu_by_id(GPU_IDS)
print(current_gpu)


import logging
import json
import glob
import datetime

from ipywidgets import interact
%matplotlib inline
%reload_ext autoreload
%autoreload 2


from src.utils.utils_io import Console_and_file_logger, init_config
from src.visualization.visualize import show_2D_or_3D
from src.data.dataset import get_img_msk_files_from_split_dir, load_acdc_files
from src.data.generators import DataGenerator, get_samples
from src.utils.unet_3d_metrics import weighted_dice_coefficient_loss
from src.models.ModelManager import get_model
from src.utils.KerasCallbacks import get_callbacks
from keras.utils import plot_model
import src.utils.my_metrics as metr


# define experiment name for report, model and log paths + filenames
EXPERIMENT = '3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4'
now = datetime.datetime.now()

# image params, change for different input data/architecture
ARCHITECTURE = '3D' # 2D
DIM = [12, 224, 224]

SPACING = [6, 1.0,1.0] # used by sitk, opposite order than numpy or tensorflow!
IMG_CHANNELS = 1
MASK_VALUES = [0, 1, 2, 3]  
MASK_CLASSES = len(MASK_VALUES)
# Background = 0 = Y[:,:,0]
# RV = 1 = Y[:,:,1] 
# Myo = 2 = Y[:,:,2] 
# LV = 3 = Y[:,:,3]
AUGMENT = False
SHUFFLE = True
AUGMENT_GRID = True
RESAMPLE = False


# path params
DATASET = 'acdc'  # 'acdc' # or 'tetra'
TRAIN_PATH = 'data/raw/ACDC/original/train/'
VAL_PATH = 'data/raw/ACDC/original/val/'
TEST_PATH = 'data/raw/ACDC/original/test/'

MODEL_PATH = os.path.join(os.path.join('models', EXPERIMENT), str(now.strftime("%Y-%m-%d_%H_%M")))
TENSORBOARD_LOG_DIR = os.path.join(os.path.join('reports/tensorboard_logs', EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
CONFIG_PATH = os.path.join(os.path.join('reports/configs/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
HISTORY_PATH = os.path.join(os.path.join('reports/history/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))

# training params
GENERATOR_WORKER = 4 # if not set, use batchsize
seed = 42
BATCHSIZE =  4 # 32, 64, 24, 16, 1
INITIAL_EPOCH = 0
EPOCHS = 150
FOLDS = 4
EPOCHS_BETWEEN_CHECKPOINTS = 5
MONITOR_FUNCTION = 'loss'
MONITOR_MODE = 'min'

# Network params
OPTIMIZER = 'Adam'  # Adam, Adagrad, RMSprop, Adadelta,  # https://keras.io/optimizers/
ACTIVATION = 'elu'  # 'elu' --> works well with binary_crossentropy and bce_dice_loss, relu does not work, it clips negative values, bse does return negative values
LEARNING_RATE = 0.001
DECAY = 0.0
EPSILON = 1e-08
DROPOUT_L1_L2 = 0.3 # best with 0.4 and other 0.5
DROPOUT_L3_L4 = 0.4
DROPOUT_L5 = 0.5
BATCH_NORMALISATION = True

#metrics = [jaccard_coef, jaccard_coef_background, jaccard_coef_rv, jaccard_coef_lv, jaccard_coef_myo]
metrics = [
    metr.dice_coef_labels,
    metr.dice_coef_myo,
    metr.dice_coef_lv,
    metr.dice_coef_rv,
]

#LOSS_FUNCTION = keras.losses.categorical_crossentropy
#weights = np.array([1,2,2,3]) # Class one at 1, class 2, 10 times the normal weights, class 3 and 4 20x.
#LOSS_FUNCTION = weighted_categorical_crossentropy(weights)
#LOSS_FUNCTION = cce_dice_loss
LOSS_FUNCTION = metr.bce_dice_jac_loss

Console_and_file_logger(EXPERIMENT, logging.INFO)


# Define a config for param injection,
# save a serialized version, 
# make sure all paths exist
config = init_config(locals(), True)

# set warnings lvl for skimage
#warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
# define a Tensorflow config
#tf_config = tf.ConfigProto()
#tf_config.gpu_options.allow_growth = True
#tf_session = tf.Session(config=tf_config)
#tf.keras.backend.set_session(tf_session)

search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12471527123894432245
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 42167337032190329
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23332631348
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 4887761710972577610
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23871111168
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 1786806131309717536
physical_device_desc: "device: 1, name: TITAN RTX, pci bus id: 0000:02:00.0, compute capability

Using TensorFlow backend.
2019-09-17 14:57:54,419 INFO -------------------- Start --------------------
2019-09-17 14:57:54,419 INFO Working directory: /mnt/data/git/cardio.
2019-09-17 14:57:54,420 INFO Log file: ./logs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4.log
2019-09-17 14:57:54,421 INFO config saved:
 {
    "ACTIVATION": "elu",
    "ARCHITECTURE": "3D",
    "AUGMENT": false,
    "AUGMENT_GRID": true,
    "BATCHSIZE": 4,
    "BATCH_NORMALISATION": true,
    "CONFIG_PATH": "reports/configs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57",
    "DATASET": "acdc",
    "DECAY": 0.0,
    "DIM": [
        12,
        224,
        224
    ],
    "DROPOUT_L1_L2": 0.3,
    "DROPOUT_L3_L4": 0.4,
    "DROPOUT_L5": 0.5,
    "EPOCHS": 150,
    "EPOCHS_BETWEEN_CHECKPOINTS": 5,
    "EPSILON": 1e-08,
    "EXPERIMENT": "3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4",
    "FOLDS": 4,
    "GENERATOR_WORKER": 4,
    "GPU_IDS": "0,1",
    "HISTORY_PATH": "r

In [22]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.wrappers.scikit_learn import KerasClassifier
import copy

import types


class KerasBatchClassifier(KerasClassifier):

    def fit(self, X, y, **train_params):
        
        config_ = copy.deepcopy(self.sk_params.get('config'))
        
        # remove validation data from train config
        x_val = train_params.pop('x_val', None)
        y_val = train_params.pop('y_val', None)
        
        print(train_params)
        config_.update(train_params)
        logging.info(sorted(config_.items()))
        self.model = self.build_fn(config=config_)

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__


        
        # create a batch generator
        batch_generator = DataGenerator(X, y, config=config_)
        config_['AUGMENT_GRID'] = False # make sure no augmentation will be applied to the validation data
        validation_generator = DataGenerator(x_val, y_val , config=config_) if x_val else DataGenerator(X, y, config=config_)
        
        callbacks=get_callbacks(config_, batch_generator, validation_generator)
        
        
        epochs = config_.get('EPOCHS', 100)
        print('epochs: {}'.format(epochs))
        print(config_['EXPERIMENT'])
        print(len(X))
        initial_epoch=1
        logging.info('fit model')

        self.__history = self.model.fit_generator(
            generator=batch_generator,
            epochs=1,
            callbacks=callbacks,
            steps_per_epoch = len(batch_generator),
            validation_data=validation_generator,
            initial_epoch=initial_epoch,
            max_queue_size=30,
            workers=8,
            #use_multiprocessing=True,
            verbose=1)

        return self.__history


    def score(self, X, y, **kwargs):
        return 1

    @property
    def history(self):
        return self.__history

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
ParameterGrid?

Init signature: ParameterGrid(param_grid)
Docstring:     
Grid of parameters with a discrete number of values for each.

Can be used to iterate over parameter value combinations with the
Python built-in function iter.

Read more in the :ref:`User Guide <search>`.

Parameters
----------
param_grid : dict of string to sequence, or sequence of such
    The parameter grid to explore, as a dictionary mapping estimator
    parameters to sequences of allowed values.

    An empty dict signifies default parameters.

    A sequence of dicts signifies a sequence of grids to search, and is
    useful to avoid exploring parameter combinations that make no sense
    or have no effect. See the examples below.

Examples
--------
>>> from sklearn.model_selection import ParameterGrid
>>> param_grid = {'a': [1, 2], 'b': [True, False]}
>>> list(ParameterGrid(param_grid)) == (
...    [{'a': 1, 'b': True}, {'a': 1, 'b': False},
...     {'a': 2, 'b': True}, {'a': 2, 'b': False}])
True

>>> grid = [{'kernel'

In [24]:
model = KerasBatchClassifier(build_fn=get_model, config=config)

x_train, y_train = load_acdc_files(config['TRAIN_PATH'])
x_val, y_val = load_acdc_files(config['VAL_PATH'])

AUGMENT_GRID   = [True, False]

param_grid = dict(AUGMENT_GRID=AUGMENT_GRID)
params_grid = list(ParameterGrid(param_grid))


In [25]:
[model.fit(x_train, y_train, verbose=1, x_val=x_val, y_val=y_val, **params) for params in params_grid]

2019-09-17 15:19:11,488 INFO [('ACTIVATION', 'elu'), ('ARCHITECTURE', '3D'), ('AUGMENT', False), ('AUGMENT_GRID', True), ('BATCHSIZE', 4), ('BATCH_NORMALISATION', True), ('CONFIG_PATH', 'reports/configs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57'), ('DATASET', 'acdc'), ('DECAY', 0.0), ('DIM', [12, 224, 224]), ('DROPOUT_L1_L2', 0.3), ('DROPOUT_L3_L4', 0.4), ('DROPOUT_L5', 0.5), ('EPOCHS', 150), ('EPOCHS_BETWEEN_CHECKPOINTS', 5), ('EPSILON', 1e-08), ('EXPERIMENT', '3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4'), ('FOLDS', 4), ('GENERATOR_WORKER', 4), ('GPU_IDS', '0,1'), ('HISTORY_PATH', 'reports/history/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57'), ('IMG_CHANNELS', 1), ('INITIAL_EPOCH', 0), ('LEARNING_RATE', 0.001), ('LOSS_FUNCTION', <function bce_dice_jac_loss at 0x7f7f9401f598>), ('MASK_CLASSES', 4), ('MASK_VALUES', [0, 1, 2, 3]), ('MODEL_PATH', 'models/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_

{'verbose': 1, 'AUGMENT_GRID': True}


2019-09-17 15:19:30,034 INFO Optimizer: Adam
2019-09-17 15:19:30,069 INFO Create DataGenerator
2019-09-17 15:19:30,070 INFO Datagenerator created with: 
 shape: [12, 224, 224]
 batchsize: 4
 Scaler: MinMax
 Images: 150 
 Augment_grid: True 
 Thread workers: 4
2019-09-17 15:19:30,071 INFO No augmentation
2019-09-17 15:19:30,071 INFO Create DataGenerator
2019-09-17 15:19:30,071 INFO Datagenerator created with: 
 shape: [12, 224, 224]
 batchsize: 4
 Scaler: MinMax
 Images: 48 
 Augment_grid: False 
 Thread workers: 4
2019-09-17 15:19:30,072 INFO No augmentation
2019-09-17 15:19:30,826 INFO feed 4 Tensorboard is ready
2019-09-17 15:19:30,836 INFO fit model


epochs: 150
3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4
150


2019-09-17 15:20:00,216 INFO [('ACTIVATION', 'elu'), ('ARCHITECTURE', '3D'), ('AUGMENT', False), ('AUGMENT_GRID', False), ('BATCHSIZE', 4), ('BATCH_NORMALISATION', True), ('CONFIG_PATH', 'reports/configs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57'), ('DATASET', 'acdc'), ('DECAY', 0.0), ('DIM', [12, 224, 224]), ('DROPOUT_L1_L2', 0.3), ('DROPOUT_L3_L4', 0.4), ('DROPOUT_L5', 0.5), ('EPOCHS', 150), ('EPOCHS_BETWEEN_CHECKPOINTS', 5), ('EPSILON', 1e-08), ('EXPERIMENT', '3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4'), ('FOLDS', 4), ('GENERATOR_WORKER', 4), ('GPU_IDS', '0,1'), ('HISTORY_PATH', 'reports/history/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57'), ('IMG_CHANNELS', 1), ('INITIAL_EPOCH', 0), ('LEARNING_RATE', 0.001), ('LOSS_FUNCTION', <function bce_dice_jac_loss at 0x7f7f9401f598>), ('MASK_CLASSES', 4), ('MASK_VALUES', [0, 1, 2, 3]), ('MODEL_PATH', 'models/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17

{'verbose': 1, 'AUGMENT_GRID': False}


2019-09-17 15:20:22,329 INFO Optimizer: Adam
2019-09-17 15:20:22,363 INFO Create DataGenerator
2019-09-17 15:20:22,364 INFO Datagenerator created with: 
 shape: [12, 224, 224]
 batchsize: 4
 Scaler: MinMax
 Images: 150 
 Augment_grid: False 
 Thread workers: 4
2019-09-17 15:20:22,364 INFO No augmentation
2019-09-17 15:20:22,364 INFO Create DataGenerator
2019-09-17 15:20:22,365 INFO Datagenerator created with: 
 shape: [12, 224, 224]
 batchsize: 4
 Scaler: MinMax
 Images: 48 
 Augment_grid: False 
 Thread workers: 4
2019-09-17 15:20:22,365 INFO No augmentation
2019-09-17 15:20:22,642 INFO feed 4 Tensorboard is ready
2019-09-17 15:20:22,651 INFO fit model


epochs: 150
3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4
150


In [68]:
def foo(**config_):
    #print(kwargs['EXPERIMENT'])
    print(config_.get('AUGMENT_GRID', 'not_given'))
    print(config_)
foo(**config)

True
{'GPU_IDS': '0,1', 'EXPERIMENT': '3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4', 'ARCHITECTURE': '3D', 'DIM': [12, 224, 224], 'SPACING': [6, 1.0, 1.0], 'IMG_CHANNELS': 1, 'MASK_VALUES': [0, 1, 2, 3], 'MASK_CLASSES': 4, 'AUGMENT': False, 'SHUFFLE': True, 'AUGMENT_GRID': True, 'RESAMPLE': False, 'DATASET': 'acdc', 'TRAIN_PATH': 'data/raw/ACDC/original/train/', 'VAL_PATH': 'data/raw/ACDC/original/val/', 'TEST_PATH': 'data/raw/ACDC/original/test/', 'MODEL_PATH': 'models/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_13_06', 'TENSORBOARD_LOG_DIR': 'reports/tensorboard_logs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_13_06', 'CONFIG_PATH': 'reports/configs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_13_06', 'HISTORY_PATH': 'reports/history/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_13_06', 'GENERATOR_WORKER': 4, 'BATCHSIZE': 4, 'INITIAL_EPOCH': 0, 'EPOCHS': 150, 'FOLDS': 4, 'EPOCHS_BETWEEN_CH

In [7]:
config

{'GPU_IDS': '0,1',
 'EXPERIMENT': '3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4',
 'ARCHITECTURE': '3D',
 'DIM': [12, 224, 224],
 'SPACING': [6, 1.0, 1.0],
 'IMG_CHANNELS': 1,
 'MASK_VALUES': [0, 1, 2, 3],
 'MASK_CLASSES': 4,
 'AUGMENT': False,
 'SHUFFLE': True,
 'AUGMENT_GRID': True,
 'RESAMPLE': False,
 'DATASET': 'acdc',
 'TRAIN_PATH': 'data/raw/ACDC/original/train/',
 'VAL_PATH': 'data/raw/ACDC/original/val/',
 'TEST_PATH': 'data/raw/ACDC/original/test/',
 'MODEL_PATH': 'models/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57',
 'TENSORBOARD_LOG_DIR': 'reports/tensorboard_logs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57',
 'CONFIG_PATH': 'reports/configs/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57',
 'HISTORY_PATH': 'reports/history/3D/acdc/bn_grid_s6_z12_1_1_1_2_Drop03_04_05_batchsize4/2019-09-17_14_57',
 'GENERATOR_WORKER': 4,
 'BATCHSIZE': 4,
 'INITIAL_EPOCH': 0,
 'EPOCHS': 150,
 'FOLDS': 4,